In [34]:
import requests
from bs4 import BeautifulSoup as bs
import pymongo
import  re

headers = {'accept': '*/*',
           'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36'}
f = open('connect_string.txt', 'r')
connect = f.read().split(":")
f.close()
db_address = connect[0]
db_port = int(connect[1])
db_name = 'avito_parse_data'
coll_name = 'raw_data'

In [43]:
def avito_parse(base_url, header):
    jobs_loc = []
    page_counter = 1
    session = requests.Session()
    request = session.get(base_url, headers=header)
    if request.status_code == 200:
        while True:
            soup = bs(request.content, 'html.parser')
            divs = soup.find_all('div', attrs={'class': 'snippet-horizontal item item_table clearfix js-catalog-item-enum item-with-contact js-item-extended'})
            for div in divs:
                title = div.find('a', attrs={'class': 'snippet-link'}).text
                price = div.find('span', attrs={'data-marker': 'item-price'}).text
                #if compensation == None:  # Если зарплата не указана
                #    compensation = ''
                #else:
                #     compensation = (div.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text).replace(u'\xa0', '')
                address = div.find('span', attrs={'class': 'item-address__string'}).text 
                all_txt = [title, price, address]
                jobs_loc.append(all_txt)

            nextPage = soup.find("span", {"data-marker": "pagination-button/next"})
            if str(nextPage).find("readonly") == -1:
                page_counter += 1
                base_url = url_base + "&p=" + str(page_counter)
                request = session.get(base_url, headers=header)
            else:
                break
        return jobs_loc
    else:
        print('error')




In [44]:
url_base = 'https://www.avito.ru/moskva/kvartiry/prodam-ASgBAgICAUSSA8YQ?metro=125&q=4-к+квартира'
url = url_base #+ 'search/vacancy?area=' + search_area + '&clusters=true&enable_snippets=true&search_period=' + search_period + '&text=' + search_value + '&page=0'  # area=1317 - Пермский край, search_period=30 - период просмотра, page=0 - Номер страницы
advert = [] 
advert = avito_parse(url, headers)

In [45]:
advert

[['4-к квартира, 213 м², 4/8 эт.',
  '\n 61 000 000  ₽\n  ',
  '\n Москва, Ленинградское шоссе, 37к1\n '],
 ['4-к квартира, 123 м², 24/24 эт.',
  '\n 25 743 900  ₽\n  ',
  '\n ш. Волоколамское, корп. 10, вл. 71/13\n '],
 ['4-к квартира, 120.6 м², 24/24 эт.',
  '\n 23 987 340  ₽\n  ',
  '\n ш. Волоколамское, корп. 10, вл. 71/13\n '],
 ['4-к квартира, 220 м², 10/30 эт.',
  '\n 64 900 000  ₽\n  ',
  '\n Москва, Авиационная улица, 79Б\n '],
 ['4-к квартира, 98.1 м², 9/25 эт.',
  '\n 17 500 000  ₽\n  ',
  '\n Волоколамское ш., 81к2\n '],
 ['4-к квартира, 152 м², 3/17 эт.',
  '\n 34 800 000  ₽\n  ',
  '\n Лодочная ул., 17\n '],
 ['4-к квартира, 113 м², 11/21 эт.',
  '\n 20 000 000  ₽\n  ',
  '\n Волоколамский пр.\n '],
 ['4-к квартира, 126 м², 40/40 эт.',
  '\n 42 000 000  ₽\n  ',
  '\n Москва, Авиационная улица, 79В\n '],
 ['4-к квартира, 98.1 м², 9/25 эт.',
  '\n 17 500 000  ₽\n  ',
  '\n Волоколамское ш., 81к2с6\n '],
 ['4-к квартира, 200 м², 4/10 эт.',
  '\n 93 660 000  ₽\n  ',
  '\n Мос

In [ ]:
def hhru_write_db(connect, port, db, coll, job_list):
    conn = pymongo.MongoClient(connect, port)
    db = conn[db]
    coll = db[coll]
    coll.remove({"_search_parr": job_list[0][5]})
    for i in job_list:
        doc = {"_job_name": i[0],
               "_salary": i[1],
               "_company": i[2],
               "_description": i[3],
               "_link": i[4],
               "_search_parr": i[5]
               }
        coll.save(doc)



In [ ]:
def salary_calc_avg(connect, port, db, coll,search_value):
    conn = pymongo.MongoClient(connect, port)
    db = conn[db]
    coll = db[coll]
    avg_salary = 0
    count = 0

    for item in coll.find({"_search_parr": search_value}):
       # print(re.sub("[^-0-9]", "", item["_salary"]).split('-'))
        avg_el = re.sub("[^-0-9]", "", item["_salary"]).split('-')
        if len(avg_el) == 2:
            avg_salary = (int(avg_el[0] or 0) + int(avg_el[1] or 0))/2 + int(avg_salary)
        else:
            avg_salary = int(avg_el[0] or 0) + int(avg_salary)
        if int(re.sub("[^0-9]", "", item["_salary"]) or 0) > 0:
            count = count + 1
   # print(avg_salary/count)
    #print(count)
    if count == 0:
        return 0
    else:
        return avg_salary/count

exit_flag = False
flag_country = False
flag_prof = False
flag_period = False

print('Вас приветствует программа для подсчёта среднего уровня зарплаты для введенной должности')
while (not exit_flag):
    if not flag_country:
        print('Введите номер региона вашего проживания (Пермский край - 1317):')
        search_area = input() #'1317'
    if not flag_prof:
        print('Введите наименование должности (или ключевое слово для поиска набора профессий):')
        search_value = input()# 'python'
    if not flag_period:
        print('Введите период, за который хотите получить информацию (одно число):')
        search_period = input() #'30'
    url_base = 'https://perm.hh.ru/'
    url = url_base + 'search/vacancy?area=' + search_area + '&clusters=true&enable_snippets=true&search_period=' + search_period + '&text=' + search_value + '&page=0'  # area=1317 - Пермский край, search_period=30 - период просмотра, page=0 - Номер страницы
    jobs = []

    jobs = hhru_parse(url, headers, search_value)
    print('Данные успешно скачаны')
    hhru_write_db(db_address, db_port, db_name, coll_name, jobs)
    print('Данные успешно обновлены в БД')
    print('Средняя заработная плата по вашему поисковому запросу: ' + str(salary_calc_avg(db_address, db_port, db_name, coll_name, search_value)))
    print('Вы хотите выполнить поиск по другой профессии? Y/N')
    q1 = input()
    if q1 == 'Y':
        flag_country = True
        flag_period = True
    else:
        print('Вы хотите выполнить поиск по всем новым параметрам? Y/N')
        q2 = input()
        if q2 == 'Y':
            flag_country = False
            flag_country = False
        else:
            exit_flag = True
    print('Работа с программой завершена')